<a href="https://colab.research.google.com/github/walt0923/NTCU-Machine-Learning/blob/assignment-%3CACS111133%3E/acs1111332.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/walt0923/NTCU-Machine-Learning


Cloning into 'NTCU-Machine-Learning'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 18 (delta 1), reused 0 (delta 0), pack-reused 16 (from 1)
Receiving objects: 100% (18/18), 5.13 KiB | 5.13 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
# 匯入必要套件
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score, accuracy_score, precision_score,
    recall_score, f1_score, roc_auc_score, confusion_matrix
)

import kagglehub  # 若未安裝，請執行 !pip install kagglehub

# 一般設定，請勿更改 TEST_SIZE
RANDOM_SEED = 42
TEST_SIZE = 0.3


In [ ]:
!pip install kagglehub


In [ ]:
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# 載入資料集 (來自 kagglehub)
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data['Class'] = data['Class'].astype(int)

# 預處理資料
data = data.drop(['Time'], axis=1)  # 移除時間欄位
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))


In [ ]:
# 分類詐騙與非詐騙交易
fraud = data[data['Class'] == 1]
nonfraud = data[data['Class'] == 0]

# 顯示數量
print(f'Fraudulent: {len(fraud)}, non-fraudulent: {len(nonfraud)}')

# 顯示詐騙樣本的比例
print(f'the positive class (frauds) percentage: {len(fraud)}/{len(fraud) + len(nonfraud)} '
      f'({len(fraud)/(len(fraud) + len(nonfraud))*100:.3f}%)')


Fraudulent: 492, non-fraudulent: 284315
the positive class (frauds) percentage: 492/284807 (0.173%)


# **1**

In [ ]:
# 特徵與標籤分開
X = np.asarray(data.loc[:, ~data.columns.isin(['Class'])])  # 所有欄位除了 Class
Y = np.asarray(data.loc[:, data.columns == 'Class'])        # 只有 Class 欄位

# 切分訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=TEST_SIZE, random_state=RANDOM_SEED
)

scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

# 建立 Random Forest 模型並訓練
rf_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED)
rf_model.fit(X_train, y_train.ravel())  # 注意 y 要扁平化


RandomForestClassifier(random_state=42)

In [ ]:
# 定義模型評估函式
def evaluation(y_true, y_pred, model_name="Model"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f'\n{model_name} Evaluation:')
    print('===' * 15)
    print(' Accuracy:', accuracy)
    print(' Precision Score:', precision)
    print(' Recall Score:', recall)
    print(' F1 Score:', f1)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


In [ ]:
# 預測與輸出結果
y_pred = rf_model.predict(X_test)
evaluation(y_test, y_pred, model_name="Random Forest")



Random Forest Evaluation:
 Accuracy: 0.9996371850239341
 Precision Score: 0.9411764705882353
 Recall Score: 0.8235294117647058
 F1 Score: 0.8784313725490196

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.94      0.82      0.88       136

    accuracy                           1.00     85443
   macro avg       0.97      0.91      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# 提取特徵與標籤
X = np.asarray(data.drop(columns=['Class']))
y = np.asarray(data['Class'])

# 分割資料集（使用 stratify=y 保持類別比例）
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y
)

# 特徵標準化
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# 從訓練資料中選取少量正常交易樣本，用於無監督訓練
n_x_train = x_train[y_train == 0]  # 選擇非詐騙交易
n_x_train = n_x_train[:1000]       # 只取前 1000 筆


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# 測試不同群數的 KMeans 並用 Silhouette Score 評估
scores = []
for k in range(2, 5):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=RANDOM_SEED)
    kmeans.fit(n_x_train)
    score = silhouette_score(n_x_train, kmeans.labels_)
    scores.append(score)

# 找出最佳群數
optimal_k = np.argmax(scores) + 2

# 用最佳群數訓練 KMeans
kmeans = KMeans(n_clusters=optimal_k, init='k-means++', random_state=RANDOM_SEED)
kmeans.fit(n_x_train)

# 對測試資料進行預測
y_pred_test = kmeans.predict(x_test)

# 標籤對齊（因為 KMeans 標籤沒有語義順序）
def align_labels(y_true, y_pred, n_clusters):
    labels = np.zeros_like(y_pred)
    for i in range(n_clusters):
        mask = (y_pred == i)
        if np.sum(mask) > 0:
            # 對每群找出對應最多的真實類別
            labels[mask] = np.bincount(y_true[mask]).argmax()
        else:
            labels[mask] = 0  # 預設為正常類別
    return labels

# 對齊標籤
y_pred_aligned = align_labels(y_test, y_pred_test, optimal_k)


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report
)

# 定義評估函式
def evaluation(y_true, y_pred, model_name="Model"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f'\n{model_name} Evaluation:')
    print('===' * 15)
    print(' Accuracy:', accuracy)
    print(' Precision Score:', precision)
    print(' Recall Score:', recall)
    print(' F1 Score:', f1)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

# 執行評估
evaluation(y_test, y_pred_aligned, model_name="KMeans (Unsupervised)")



KMeans (Unsupervised) Evaluation:
 Accuracy: 0.9987242957293166
 Precision Score: 0.782608695652174
 Recall Score: 0.36486486486486486
 F1 Score: 0.4976958525345622

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.78      0.36      0.50       148

    accuracy                           1.00     85443
   macro avg       0.89      0.68      0.75     85443
weighted avg       1.00      1.00      1.00     85443



# **2**

In [ ]:
X = np.asarray(data.loc[:, ~data.columns.isin(['Class'])])
Y = np.asarray(data.loc[:, data.columns == 'Class'])
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=TEST_SIZE, random_state=RANDOM_SEED
)
# scaler = StandardScaler()
# x_train = scaler.fit_transform(X_train)
# x_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import IsolationForest

iso_model = IsolationForest(contamination=0.02, random_state=RANDOM_SEED)
iso_model.fit(x_train)

iso_score_train = iso_model.decision_function(x_train).reshape(-1, 1)
iso_score_test = iso_model.decision_function(x_test).reshape(-1, 1)

train_anomaly = iso_model.predict(x_train)
test_anomaly = iso_model.predict(x_test)
train_anomaly = (train_anomaly == -1).astype(int).reshape(-1, 1)
test_anomaly = (test_anomaly == -1).astype(int).reshape(-1, 1)


X_train_aug = np.hstack((x_train, iso_score_train, train_anomaly))
X_test_aug = np.hstack((x_test, iso_score_test, test_anomaly))

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=RANDOM_SEED,
    learning_rate=0.15,
    )
xgb_model.fit(X_train_aug, y_train)

y_pred = xgb_model.predict(X_test_aug)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:36:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report
)

def evaluation(y_true, y_pred, model_name="Model"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f'\n{model_name} Evaluation:')
    print('===' * 15)
    print(' Accuracy:', accuracy)
    print(' Precision Score:', precision)
    print(' Recall Score:', recall)
    print(' F1 Score:', f1)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


In [ ]:
evaluation(y_test, y_pred, model_name="isoXGB")



isoXGB Evaluation:
 Accuracy: 0.9996605924417448
 Precision Score: 0.9421487603305785
 Recall Score: 0.8382352941176471
 F1 Score: 0.8871595330739299

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.94      0.84      0.89       136

    accuracy                           1.00     85443
   macro avg       0.97      0.92      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [1]:
!git config --global user.email "walt920923@gmail.com"
!git config --global user.name "walt0923"

# 登入一次性設定 HTTPS access（用 token 替代密碼）
# ⚠️ 安全建議：不要硬編碼在 Colab，使用 input()


In [2]:
!git clone https://github.com/walt0923/NTCU-Machine-Learning.git


Cloning into 'NTCU-Machine-Learning'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 34 (delta 5), reused 0 (delta 0), pack-reused 16 (from 1)
Receiving objects: 100% (34/34), 22.41 KiB | 7.47 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [3]:
%cd NTCU-Machine-Learning


/content/NTCU-Machine-Learning


In [4]:
!git checkout -b assignment-ACS111133


Switched to a new branch 'assignment-ACS111133'


In [5]:
!git add .
!git commit -m "Submit assignment for 109123456"
!git push origin assignment-ACS111133


On branch assignment-ACS111133
nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
